In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import yaml
import copy
from utils import extract_frame


In [ ]:
filename = r'.\data\data_1\data\0000000150.png'
camera_file =  r'.\data\data_1\camera_params.yaml'
with open(camera_file) as file:
    camera_coeffs = yaml.load(file, Loader=yaml.FullLoader)
K = np.array(list(float(i) for i in (camera_coeffs['K']).split())).reshape(3,3)
D = np.array(list(float(i) for i in (camera_coeffs['D']).split()))
C = cv2.VideoCapture('challenge_video.mp4')


In [ ]:
def histogram(image):
    # Calculating the histogram
    hist = np.sum(image[image.shape[0]//2:,:], axis=0)
    return hist

In [ ]:
def sliding_window(histogram, sobel):

        # Create an output image to draw on and visualize the result
        output_image = np.dstack((sobel, sobel, sobel))*255
       
        # We need to find the peak of eft and right half of histogram which should be the starting point for left and right line
        mid_point = np.int(histogram.shape[0]/2)
        start_left_x = np.argmax(histogram[:mid_point])
        start_right_x = np.argmax(histogram[mid_point:]) + mid_point
        
 # Identify the x and y positions of all nonzero pixels in the image

        lane_detected = sobel.nonzero()
        lane_detected_y = np.array(lane_detected[0])
        lane_detected_x = np.array(lane_detected[1])
        
        
        # Choose the number of sliding windows
        image_split = 30
        # Set height of windows
        window_height = np.int(sobel.shape[0]/image_split)
        
        

        
        # Current positions to be updated for each window
        lx_present = start_left_x
        rx_present = start_right_x
        
        
        
        # Declared margin 
        margin = 60
        # Set minimum number of pixels found to recenter window
        minimum_pixels = 60
        
        
        
        # List to red the left and right lane pixel indices
        index_left = []
        index_right = []

        
        
        # Step through the windows one by one
        for window in range(image_split):         
            
            # Identify window boundaries in x and y (and right and left)
            window_y_low = sobel.shape[0] - (window+1)*window_height
            window_y_high = sobel.shape[0] - window*window_height
            window_x_left_low = lx_present - margin
            window_x_left_high = lx_present + margin
            window_x_right_low = rx_present - margin
            window_x_right_high = rx_present + margin
                    
            
            # Drawing the windows on the image
            cv2.rectangle(output_image,(window_x_left_low,window_y_low),(window_x_left_high,window_y_high),(255,255,255), 2) 
            cv2.rectangle(output_image,(window_x_right_low,window_y_low),(window_x_right_high,window_y_high),(255,255,255), 2) 
            
            
            # Identify the nonzero pixels in x and y within the window
            correct_left = ((lane_detected_y >= window_y_low) & (lane_detected_y < window_y_high) & (lane_detected_x >= window_x_left_low) & (lane_detected_x < window_x_left_high)).nonzero()[0]
            correct_right = ((lane_detected_y >= window_y_low) & (lane_detected_y < window_y_high) & (lane_detected_x >= window_x_right_low) & (lane_detected_x < window_x_right_high)).nonzero()[0]
            
                      
            # Append these indices to the lists
            index_left.append(correct_left)
            index_right.append(correct_right)
            
            
            # Recentering of the next window
            if len(correct_left) > minimum_pixels:
                lx_present = np.int(np.mean(lane_detected_x[correct_left]))
            if len(correct_right) > minimum_pixels:        
                rx_present = np.int(np.mean(lane_detected_x[correct_right]))

                              
        # Concatenate the arrays of indices
        index_left = np.concatenate(index_left)
        index_right = np.concatenate(index_right)

          
        # Extract left and right line pixel positions
        lx = lane_detected_x[index_left]
        ly = lane_detected_y[index_left] 
        rx = lane_detected_x[index_right]
        ry = lane_detected_y[index_right] 
        return lx, ly, rx, ry, index_left, index_right

In [ ]:
while(True):
    ret, image = C.read()
    if ret == True:
        undistorted = cv2.undistort(image, K, D)
        #gray = cv2.cvtColor(undistorted,cv2.COLOR_BGR2GRAY)
        gauss = cv2.GaussianBlur(undistorted,(7,7),1)
        destination_1 = np.array([[0, 0],[300, 0],[0,300],[300, 300]], dtype = "float32")
        source_1 = np.array([[560,450],[740, 450],[95, 710],[1260, 710]], dtype = "float32")
#         H = homography(source_1, destination_1)
        h, status = cv2.findHomography(source_1, destination_1)

        warp = cv2.warpPerspective(gauss, h, (255, 255))
        channel = warp[:,:,2]               #Selecting the red channel of the image
        ret, thresh = cv2.threshold(channel, 110, 255, cv2.THRESH_BINARY)               #Thresholding the image
        sobelx = cv2.Sobel(thresh, cv2.CV_64F, 1, 0)
        sobely = cv2.Sobel(thresh, cv2.CV_64F, 0, 1)
        sobel = np.sqrt((sobelx*2) + (sobely*2)) 
        get_histogram = histogram(sobel)
        lx, ly, rx, ry, index_left, index_right=sliding_window(get_histogram, sobel)
        plt.hist(get_histogram.ravel(),256,[0,256]); 
        plt.show()
        cv2.imshow('sobel', sobel)
        k = cv2.waitKey(1)
        if k == 27:
            break
        else:
            break

C.release()        
# cv2.destroyAllWindows()
